In [ ]:
# adaptation of CAN_P.m by Carl Tape
# coding by: Yongki Aiman, Tomy Gunawan, Angel Ling

# analyzing the Sumatra earthquake P wave
# custom function: wf_fft

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import warnings

from matplotlib.mlab import detrend
from obspy.core import UTCDateTime
from obspy.clients.fdsn import Client

from lib_seis import wf_fft

In [ ]:
# script settings

warnings.filterwarnings('ignore')
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = 10, 6
plt.rcParams['lines.linewidth'] = 0.5

In [ ]:
# getting waveforms (all three components)

client = Client("IRIS")

# 2004 Sumatra-Andaman earthquake
origintime = UTCDateTime("2004-12-26T00:58:53")
starttime  = origintime - ( 1 * 60 * 60 )           # 1 hour before origin time
endtime    = origintime + ( 3 * 60 * 60)            # 3 hours after origin time

st = client.get_waveforms("G", "CAN", "*", "B*", starttime, endtime, attach_response=True)
print(st)

In [ ]:
# plotting all three components of the seismogram

can   = st.copy()

can_e = can.select(component="E")
can_n = can.select(component="N")
can_z = can.select(component="Z")

t     = can[0].times()

tr_e  = can_e[0].data
tr_n  = can_n[0].data
tr_z  = can_z[0].data

title_e = f'{can_e[0].stats["station"]} ({can_e[0].stats["channel"]}) − starting {can_e[0].stats["starttime"]}'
title_n = f'{can_n[0].stats["station"]} ({can_n[0].stats["channel"]}) − starting {can_n[0].stats["starttime"]}'
title_z = f'{can_z[0].stats["station"]} ({can_z[0].stats["channel"]}) − starting {can_z[0].stats["starttime"]}'

#axx = [4500,5500]
fig, (ax1,ax2,ax3) = plt.subplots(3,1)
fig.set_size_inches(10,16)

ax1.plot(t, tr_e, color='b')
ax1.set(xlabel='seconds', ylabel='counts', title=title_e)
#ax1.set_xlim(axx)

ax2.plot(t, tr_n, color='b')
ax2.set(xlabel='seconds', ylabel='counts', title=title_n)
#ax2.set_xlim(axx)

ax3.plot(t, tr_z, color='b')
ax3.set(xlabel='seconds', ylabel='counts', title=title_z)
#ax3.set_xlim(axx)

In [ ]:
# Z component only

plt.plot(t, tr_z, color='b')
plt.title(title_z)
plt.xlabel('seconds')
plt.ylabel('counts')

In [ ]:
# extracting a subset time series of the Z component containing the P wave

st1 = st.select(component="Z").copy()

t1 = starttime + 4000                # change these to 'pick' the P wave
t2 = starttime + 6000                # change these to 'pick' the P wave

st1.trim(t1,t2)
print(st1)

t  = st1[0].times()
tr = st1[0].data

title = f'{st1[0].stats["station"]} {st1[0].stats["channel"]} − starting {st1[0].stats["starttime"]}'

plt.plot(t, tr, color='b')
plt.title(title)
plt.xlabel('seconds')
plt.ylabel('counts')

In [ ]:
# do spectral analysis here

sampling_frequency = st1[0].stats.sampling_rate
Nyquist_frequency  = sampling_frequency / 2